<a href="https://colab.research.google.com/github/jrebull/NLP/blob/main/Team18_MNA_NLP_semana_10_Actividad_10_3_modelosLLM_IA_Trabajo_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **10.3 Actividad: modelos LLM y IA en tu lugar de trabajo.**

# **Conclusiones:**





None

# **Fin de la actividad 10.3 Actividad: modelos LLM y IA en tu lugar de trabajo**